In [1]:
from qiskit import *

In [ ]:
def prepare_CCZ() -> QuantumCircuit:
    """ Prepare a 3 qubits CCZ state.
    """
    reg = QuantumRegister(3)
    qc = QuantumCircuit(reg)
    qc.h(reg)
    qc.ccz(reg[0], reg[1], reg[2])
    return qc

In [ ]:
def prepare_i_state(num: int) -> QuantumCircuit:
    """ Prepare the |i> = (|0> + i|1>)/sqrt(2) state.
    """
    qc = QuantumCircuit(num)
    for i in range(num):
        qc.h(i)
        qc.s(i)
    return qc

In [ ]:
def prepare_semiauto_magic_state() -> QuantumCircuit:
    reg_ccz = QuantumRegister(3, 'ccz')
    reg_i = QuantumRegister(3, 'i')
    reg_anc = QuantumRegister(1, 'ancilla')
    qc = QuantumCircuit(reg_ccz, reg_i, reg_anc)

    #x parity check then cz
    qc.h(reg_i[0])
    qc.cx(reg_i[0], reg_anc[0])
    qc.cz(reg_anc[0], reg_ccz[0])
    qc.cx(reg_i[0], reg_anc[0])
    qc.h(reg_i[0])

    #z parity check then cx
    qc.ccx(reg_ccz[0], reg_ccz[2], reg_anc[0])
    qc.cx(reg_anc[0], reg_i[1])
    qc.ccx(reg_ccz[0], reg_ccz[2], reg_anc[0])

    #z parity check then cx
    qc.ccx(reg_ccz[0], reg_ccz[1], reg_anc[0])
    qc.cx(reg_anc[0], reg_i[2])
    qc.ccx(reg_ccz[0], reg_ccz[1], reg_anc[0])

    qc.h(reg_ccz[0])
    return qc

In [ ]:
def z_parity_check(num: int) -> QuantumCircuit:
    """
    Sets anc qubit to the Z parity of the data qubits.
    """
    reg_data = QuantumRegister(num, 'data')
    reg_anc = QuantumRegister(1, 'ancilla')
    qc = QuantumCircuit(reg_data, reg_anc)

    for i in range(num):
        qc.cx(reg_data[i], reg_anc[0])
    return qc

In [ ]:
def x_parity_check(num: int) -> QuantumCircuit:
    """
    Sets anc qubit to the X parity of the data qubits.
    """
    reg_data = QuantumRegister(num, 'data')
    reg_anc = QuantumRegister(1, 'ancilla')
    qc = QuantumCircuit(reg_data, reg_anc)

    for i in range(num):
        qc.h(reg_data[i])
        qc.cx(reg_data[i], reg_anc[0])
        qc.h(reg_data[i])
    return qc

In [ ]:
def interact() -> QuantumCircuit:
    reg_inputs = QuantumRegister(5, 'input')
    reg_ccz = QuantumRegister(3, 'ccz')
    reg_anc = QuantumRegister(1, 'ancilla')
    reg_c = ClassicalRegister(2, 'classical')

    qc = QuantumCircuit(reg_inputs, reg_ccz, reg_anc, reg_c)

    qc.append(z_parity_check(5), reg_inputs[:] + [reg_anc[0]])
    qc.cx(reg_anc[0], reg_ccz[0])
    qc.append(z_parity_check(5).inverse(), reg_inputs[:] + [reg_anc[0]])

    qc.append(z_parity_check(2), reg_inputs[1:3] + [reg_anc[0]])
    qc.cx(reg_anc[0], reg_ccz[1])
    qc.append(z_parity_check(2).inverse(), reg_inputs[1:3] + [reg_anc[0]])

    qc.append(z_parity_check(2), reg_inputs[0:1] + reg_inputs[2:3] + [reg_anc[0]])
    qc.cx(reg_anc[0], reg_ccz[2])
    qc.append(z_parity_check(2).inverse(), reg_inputs[0:1] + reg_inputs[2:3] + [reg_anc[0]])

    qc.measure(reg_ccz[1:3], reg_c)

    return qc

In [ ]:
from qiskit.circuit.classical import expr

def reaction_limited_basis_choice() -> QuantumCircuit:
    reg_c = ClassicalRegister(2, 'ccz_post_measure')
    reg_i = QuantumRegister(3, 'i')
    reg_ci = ClassicalRegister(3, 'i_post_measure')
    qc = QuantumCircuit(reg_c, reg_i, reg_ci)

    condition = expr.bit_xor(reg_c[0], reg_c[1])
    with qc.if_test(condition):
        qc.h(reg_i[0])
    condition = reg_c[0]
    with qc.if_test(condition):
        qc.h(reg_i[1])
    condition = reg_c[1]
    with qc.if_test(condition):
        qc.h(reg_i[2])
    
    qc.h(reg_i)
    qc.measure(reg_i, reg_ci)

    return qc